# Lab - Customer analysis case study

## Day 1

In [2]:
import pandas as pd
import numpy as np
import statistics as stats
import math

**Aggregate the data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them.**

In [3]:
df1 = pd.read_csv('Data/file1.csv')
df2 = pd.read_csv('Data/file2.csv')
df3 = pd.read_csv('Data/file3.csv')

In [4]:
df1.rename(columns={'GENDER': 'Gender', 'ST': 'State'}, inplace=True)
df2.rename(columns={'GENDER': 'Gender', 'ST': 'State'}, inplace=True)

df = pd.concat([df1, df2, df3], axis=0)

**Standardizing header names**

In [5]:
def lower_case_columns(df):
    df.columns = [i.lower() for i in df.columns]
    
    return df

#or this:
#df.rename(str.lower, axis=1)

In [6]:
lower_case_columns(df)

,customer,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


**Removing duplicates**

In [7]:
len(df[df.duplicated()]), len(df[df.duplicated(keep=False)])

(2939, 2943)

In [8]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True)

,customer,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
9130,LA72316,California,M,Bachelor,23405.98798,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
9131,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
9132,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
9133,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


**Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data**

In [9]:
df.drop(columns=['customer'], inplace=True)

**Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.**

In [10]:
def remove_percentage(x):
    if isinstance(x, str):
        return x.replace('%', '')
    return x

df['customer lifetime value'] = df['customer lifetime value'].apply(remove_percentage)

In [11]:
df['customer lifetime value'].apply(type).value_counts()

<class 'float'>    7078
<class 'str'>      2057
Name: customer lifetime value, dtype: int64

In [12]:
df['customer lifetime value'] =  pd.to_numeric(df['customer lifetime value'], errors='coerce')
df['customer lifetime value'] =  df['customer lifetime value'].apply(lambda x: math.trunc(x) if np.isnan(x) != True else 0)


**clean the number of open complaints and extract the middle number which is changing between records.**

In [13]:
df['number of open complaints'].unique()

array(['1/0/00', '1/2/00', '1/1/00', '1/3/00', '1/5/00', '1/4/00', nan, 0,
       2, 3, 1, 5, 4], dtype=object)

In [14]:
df['number of open complaints'].apply(type).value_counts()

<class 'int'>      7070
<class 'str'>      2064
<class 'float'>       1
Name: number of open complaints, dtype: int64

In [15]:
def clean_complaints(x):
    if isinstance(x, str):
        return x.replace('/', '')[1]
    return x

df['number of open complaints'] = df['number of open complaints'].apply(clean_complaints)


In [16]:
df['number of open complaints'] =  pd.to_numeric(df['number of open complaints'], errors='coerce')
df['number of open complaints'] =  df['number of open complaints'].apply(lambda x: 0 if np.isnan(x) == True else int(x))


**Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns**

In [17]:
def clean_state(state):
    if state == 'Cali':
        return 'California'
    elif state == 'AZ':
        return 'Arizona'
    elif state == 'WA':
        return 'Washington'
    else:
        return state

def clean_gender(gender):
    if gender in ['F', 'Femal', 'female', 'Female']:
        return 'Female'
    elif gender in ['M', 'Male']:
        return 'Male'
    else:
        return gender

df['state'] = list(map(clean_state, df['state']))
df['gender'] = list(map(clean_gender, df['gender']))

## Day 2

**Replacing null values – Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values) Hint: numpy.nan is considered of float64 data type.**

In [18]:
nonnan_values = {'state': 'Unspecified',
                'gender': 'Unspecified',
                'education': 'Unspecified',
                'income': round(np.nanmean(df['income'])),
                'monthly premium auto': round(np.nanmean(df['monthly premium auto'])),
                'policy type': 'Unspecified',
                'vehicle class': 'Unspecified',
                'total claim amount': round(np.nanmean(df['total claim amount']))}

df.fillna(value=nonnan_values, inplace=True)

In [19]:
df['income'] =  df['income'].apply(lambda x: math.trunc(x))
df['monthly premium auto'] =  df['monthly premium auto'].apply(lambda x: math.trunc(x))
df['total claim amount'] =  df['total claim amount'].apply(lambda x: math.trunc(x))


**Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central**

In [20]:
def state_to_zone(state):
    if state == 'California':
        return 'West'
    elif state == 'Oregon':
        return 'North West'
    elif state == 'Washington':
        return 'East'
    elif state == 'Arizona' or state == 'Nevada':
        return 'Central'
    else:
        return state

df['state'] = list(map(state_to_zone, df['state']))

In [21]:
df.rename(columns={'state': 'region'}, inplace=True)

**(Optional) In the column Vehicle Class, merge the two categories Luxury SUV and Luxury Car into one category named Luxury Vehicle**

In [22]:
def clean_vehicle_class(vehicle):
    if vehicle == 'Luxury SUV' or vehicle == 'Luxury Car':
        return 'Luxury Vehicle'
    return vehicle

df['vehicle class'] = list(map(clean_vehicle_class, df['vehicle class']))
df['vehicle class'].unique()

array(['Four-Door Car', 'Two-Door Car', 'SUV', 'Luxury Vehicle',
       'Sports Car', 'Unspecified'], dtype=object)

**(Optional) Removing outliers using 1.5*IQR technique for all numerical columns.**

In [23]:
median_lifetime = stats.median(df['customer lifetime value'])
lifetime_q1 = stats.quantiles(df['customer lifetime value'])[0]
lifetime_q3 = stats.quantiles(df['customer lifetime value'])[2]
lifetime_iqr = lifetime_q3 - lifetime_q1
lifetime_lf = lifetime_q1 - 1.5*lifetime_iqr #lower fence
lifetime_uf = lifetime_q3 + 1.5*lifetime_iqr #upper fence

def remove_outliers_lifetime(x):
    if x < lifetime_lf or x > lifetime_uf:
        return median_lifetime
    return x

df['customer lifetime value'] = list(map(remove_outliers_lifetime, df['customer lifetime value']))

In [24]:
median_income = stats.median(df['income'])
income_q1 = stats.quantiles(df['income'])[0]
income_q3 = stats.quantiles(df['income'])[2]
income_iqr = income_q3 - income_q1
income_lf = income_q1 - 1.5*income_iqr #lower fence
income_uf = income_q3 + 1.5*income_iqr #upper fence

def remove_outliers_income(x):
    if x < income_lf or x > income_uf:
        return median_income
    return x

df['income'] = list(map(remove_outliers_income, df['income']))

In [25]:
median_premium = stats.median(df['monthly premium auto'])
premium_q1 = stats.quantiles(df['monthly premium auto'])[0]
premium_q3 = stats.quantiles(df['monthly premium auto'])[2]
premium_iqr = premium_q3 - premium_q1
premium_lf = premium_q1 - 1.5*premium_iqr #lower fence
premium_uf = premium_q3 + 1.5*premium_iqr #upper fence

def remove_outliers_premium(x):
    if x < premium_lf or x > premium_uf:
        return median_premium
    return x

df['monthly premium auto'] = list(map(remove_outliers_premium, df['monthly premium auto']))

In [26]:
#Q1-Q3 = 0 

# median_complaints = stats.median(df['number of open complaints'])
# complaints_q1 = stats.quantiles(df['number of open complaints'])[0]
# complaints_q3 = stats.quantiles(df['number of open complaints'])[2]
# complaints_iqr = complaints_q3 - complaints_q1
# complaints_lf = complaints_q1 - 1.5*complaints_iqr #lower fence
# complaints_uf = complaints_q3 + 1.5*complaints_iqr #upper fence

# def remove_outliers_complaints(x):
#     if x < complaints_lf or x > complaints_uf:
#         return median_complaints
#     return x

# df['number of open complaints'] = list(map(remove_outliers_complaints, df['number of open complaints']))

In [27]:
median_claim = stats.median(df['total claim amount'])
claim_q1 = stats.quantiles(df['total claim amount'])[0]
claim_q3 = stats.quantiles(df['total claim amount'])[2]
claim_iqr = claim_q3 - claim_q1
claim_lf = claim_q1 - 1.5*claim_iqr #lower fence
claim_uf = claim_q3 + 1.5*claim_iqr #upper fence

def remove_outliers_claim(x):
    if x < claim_lf or x > claim_uf:
        return median_claim
    return x

df['total claim amount'] = list(map(remove_outliers_claim, df['total claim amount']))

**(Optional) Standardizing the data – Use string functions to standardize the text data (lower case)**

In [28]:
df['region'] = df['region'].str.lower()
df['gender'] = df['gender'].str.lower()
df['education'] = df['education'].str.lower()
df['policy type'] = df['policy type'].str.lower()
df['vehicle class'] = df['vehicle class'].str.lower()